In [1]:
import time
import gdown
import pandas as pd
import numpy as np
from datetime import date
import yfinance as yf
import plotly.express as px
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:,.2f}'.format 

/opt/conda/envs/markets/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


# Question 1

In [2]:
url = "https://stockanalysis.com/ipos/withdrawn/"
df = pd.read_html(url)[0]
df.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [3]:
def create_company_class(df):
    df['company_class_helper'] = df['Company Name'].str.lower()
    # Conditions
    cond = [df['company_class_helper'].str.contains("acquisition|corporation|corp."),
            df['company_class_helper'].str.contains("inc.|incorporated"),
            df['company_class_helper'].str.contains("group"),
            df['company_class_helper'].str.contains("ltd.?|limited"),
            df['company_class_helper'].str.contains("holdings"),]
    choices = ["Acq.Corp", "Inc", "Group", "Limited", "Holdings"]
    df["Company Class"] = np.select(cond, choices, default="Other")
    # Drop helper column
    df.drop('company_class_helper', axis=1, inplace=True)
    return df

In [4]:
def create_avg_price(data):
    df = data.copy()
    df['Price Range'] = df['Price Range'].str.split("-")
    def compute_mean(row):
        try:
            num_list = [float(num.replace("$", "").strip()) for num in row]
            return np.mean(num_list)
        except:
            return None
    df['Avg. Price'] = df['Price Range'].apply(compute_mean)
    return df


In [5]:
df = create_company_class(df)

In [6]:
df = create_avg_price(df)

In [7]:
df.head(10)

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,ODTX,"Odyssey Therapeutics, Inc.","[, ]",-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.","[$3.00 , $4.00]",2000000,Inc,3.50
2,AURN,"Aurion Biotech, Inc.","[, ]",-,Inc,NaN
3,ROTR,"PHI Group, Inc.","[, ]",-,Inc,NaN
4,ONE,One Power Company,"[, ]",-,Other,NaN
5,HPOT,The Great Restaurant Development Holdings Limited,"[$4.00 , $6.00]",1400000,Limited,5.00
6,CABR,"Caring Brands, Inc.",[$4.00],750000,Inc,4.00
7,SQVI,"Sequoia Vaccines, Inc.","[$8.00 , $10.00]",2775000,Inc,9.00
8,SNI,Shenni Holdings Limited,"[$4.00 , $6.00]",3000000,Limited,5.00
9,KMCM,Key Mining Corp.,[$2.25],4444444,Acq.Corp,2.25


In [8]:
df['Shares Offered'] =  pd.to_numeric(df['Shares Offered'], errors='coerce')
df['Withdrawn Value'] = df['Avg. Price'] * df['Shares Offered']

In [9]:
df.notnull().sum()

Symbol             100
Company Name       100
Price Range        100
Shares Offered      72
Company Class      100
Avg. Price          73
Withdrawn Value     71
dtype: int64

In [10]:
agg_df = df.groupby("Company Class", as_index=False)['Withdrawn Value'].sum().sort_values('Withdrawn Value', ascending=False)
agg_df['Withdrawn Value'] = agg_df['Withdrawn Value']/1e6
agg_df

,Company Class,Withdrawn Value
0,Acq.Corp,"4,111.85"
2,Inc,"2,257.16"
4,Other,752.07
3,Limited,549.73
1,Group,33.79


# Question 2

In [11]:
url = "https://stockanalysis.com/ipos/2024/"
df_sharpe = pd.read_html(url)[0]
df_sharpe.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$5.00,25.00%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$26.57,564.25%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.62,-84.40%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.28,2.80%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.21,2.10%


In [12]:
df_sharpe['IPO Date'] = pd.to_datetime(df_sharpe['IPO Date'], format='mixed')

In [13]:
df_sharpe_subset = df_sharpe[df_sharpe['IPO Date'] < '2024-06-01'].copy()
df_sharpe_subset.shape, df_sharpe_subset['Symbol'].nunique()

((77, 6), 77)

In [14]:
invalid_tickers = ['NAKA', 'SUPX']
df_sharpe_subset = df_sharpe_subset.query("Symbol not in @invalid_tickers")
df_sharpe_subset.shape, df_sharpe_subset['Symbol'].nunique()

((75, 6), 75)

In [15]:
df_sharpe_subset['IPO Date'].describe()

count                     75
mean     2024-03-16 04:09:36
min      2024-01-09 00:00:00
25%      2024-02-06 12:00:00
50%      2024-03-21 00:00:00
75%      2024-04-22 00:00:00
max      2024-05-23 00:00:00
Name: IPO Date, dtype: object

In [16]:
tickers = df_sharpe_subset['Symbol'].unique().tolist()

In [17]:

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(tickers):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [18]:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

In [19]:
trading_day = stocks_df[stocks_df['Date']==date(2025, 6, 6)].copy()
trading_day.shape

(75, 27)

In [20]:
trading_day[['Sharpe', 'growth_252d']].notnull().sum()

Sharpe         71
growth_252d    71
dtype: int64

In [21]:
trading_day[['Sharpe', 'growth_252d']].describe()

,Sharpe,growth_252d
count,71.00,71.00
mean,0.29,1.15
std,0.52,1.41
min,-0.08,0.02
25%,0.04,0.29
50%,0.08,0.76
75%,0.31,1.36
max,2.84,8.10


In [22]:
top_10_sharpe = trading_day.sort_values("Sharpe", ascending=False)[['Ticker']].head(10)

In [23]:
top_10_growth_yoy =trading_day.sort_values("growth_252d", ascending=False)[['Ticker']].head(10)

In [24]:
set(top_10_sharpe['Ticker'].values).intersection(set(top_10_growth_yoy['Ticker'].values))

{'JL'}

In [25]:
set(top_10_sharpe['Ticker'].values).difference(set(top_10_growth_yoy['Ticker'].values))

{'BKHA', 'DYCQ', 'HLXB', 'IBAC', 'INTJ', 'JVSA', 'LEGT', 'MNDR', 'TRSG'}

# Question 3

In [26]:
def compute_future_growth(row, days):
    return row.shift(-days) /row

In [27]:
days = [(f"future_growth_{i}m", 21* i) for i in range(1, 13)]

In [28]:
for col, day in days:
    stocks_df[col] = stocks_df.groupby("Ticker")['Close'].transform(lambda x: compute_future_growth(x, day))

In [29]:
selected_cols = ['Close', 'Ticker', 'Year', 'Month', 'Weekday', 'Date'] + [x for x, _ in days]

In [30]:
min_date_df = stocks_df.groupby('Ticker', as_index=False)['Date'].min()

In [31]:
joined_df = stocks_df.merge(min_date_df, on=['Ticker', 'Date'], how='inner')[selected_cols]

In [32]:
joined_df.head()

,Close,Ticker,Year,Month,Weekday,Date,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,23.80,BOW,2024,5,3,2024-05-23,1.02,1.14,1.30,1.17,1.24,1.46,1.44,1.36,1.39,1.72,1.70,1.53
1,22.29,HDL,2024,5,4,2024-05-17,0.78,0.75,0.67,0.64,0.72,0.72,1.26,1.11,1.03,1.19,0.91,1.00
2,10.04,RFAI,2024,7,4,2024-07-05,1.00,1.01,1.01,1.01,1.01,1.02,1.02,1.03,1.04,1.04,1.05,NaN
3,4.07,JDZG,2024,5,2,2024-05-15,0.24,0.21,0.12,0.16,0.27,0.20,0.14,0.18,0.17,0.15,0.12,0.14
4,4.23,RAY,2024,5,2,2024-05-15,0.84,0.78,0.37,0.46,0.40,0.41,0.40,0.29,0.30,0.26,0.60,0.43


In [33]:
joined_df[[x for x, _ in days]].describe().T

,count,mean,std,min,25%,50%,75%,max
future_growth_1m,75.00,0.93,0.35,0.10,0.78,0.98,1.05,2.65
future_growth_2m,75.00,0.94,0.57,0.07,0.69,1.00,1.15,4.87
future_growth_3m,75.00,0.83,0.41,0.06,0.51,0.93,1.07,2.04
future_growth_4m,75.00,0.83,0.40,0.05,0.52,0.91,1.13,1.61
future_growth_5m,75.00,0.80,0.49,0.05,0.45,0.82,1.02,3.21
future_growth_6m,75.00,0.86,0.65,0.06,0.38,0.80,1.09,3.67
future_growth_7m,75.00,0.85,0.71,0.05,0.30,0.84,1.11,5.12
future_growth_8m,75.00,0.83,0.76,0.04,0.21,0.81,1.08,5.17
future_growth_9m,75.00,0.88,0.94,0.03,0.23,0.82,1.05,6.76
future_growth_10m,74.00,0.92,0.91,0.04,0.24,0.77,1.20,5.35


# Question 4

In [34]:

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=eee0c211-84d6-49a4-a60d-c7d5027a709c
To: /workspaces/stock-markets-zoomcamp/hw2/data.parquet
100%|██████████| 130M/130M [00:02<00:00, 61.8MB/s] 


In [42]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
].copy()

In [43]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

In [44]:
net_income/1000

np.float64(24.295523125248387)

# Plot Volatility vs. RSI

In [45]:
ticker_sector_map = {
    # U.S. Tech
    'MSFT': 'Information Technology',
    'AAPL': 'Information Technology',
    'GOOG': 'Communication Services',
    'NVDA': 'Information Technology',
    'AMZN': 'Consumer Discretionary',
    'META': 'Communication Services',

    # U.S. Financials / Healthcare / Industrials
    'BRK-B': 'Financials',
    'LLY': 'Health Care',
    'AVGO': 'Information Technology',
    'V': 'Information Technology',
    'JPM': 'Financials',
    'NVO': 'Health Care',

    # European Stocks
    'MC.PA': 'Consumer Discretionary',      # LVMH
    'ASML': 'Information Technology',
    'RMS.PA': 'Consumer Discretionary',     # Hermès
    'OR.PA': 'Consumer Staples',            # L'Oréal
    'SAP': 'Information Technology',
    'ACN': 'Information Technology',
    'TTE': 'Energy',                        # TotalEnergies
    'SIE.DE': 'Industrials',                # Siemens
    'IDEXY': 'Industrials',                 # Idex Corporation
    'CDI.PA': 'Consumer Discretionary',     # Christian Dior

    # Indian Stocks
    'RELIANCE.NS': 'Energy',
    'TCS.NS': 'Information Technology',
    'HDB': 'Financials',                    # HDFC Bank
    'BHARTIARTL.NS': 'Communication Services',
    'IBN': 'Financials',                    # ICICI Bank
    'SBIN.NS': 'Financials',                # State Bank of India
    'LICI.NS': 'Financials',                # Life Insurance Corp
    'INFY': 'Information Technology',
    'ITC.NS': 'Consumer Staples',
    'HINDUNILVR.NS': 'Consumer Staples',
    'LT.NS': 'Industrials'                  # Larsen & Toubro
}

In [46]:
selected_df['sector'] = selected_df['Ticker'].map(ticker_sector_map)

In [55]:
df_plot = selected_df.groupby(['sector'], as_index=False).agg({'volatility':'mean',
                                                               'rsi':'mean'})

In [56]:
df_plot

,sector,volatility,rsi
0,Communication Services,191.39,22.46
1,Consumer Discretionary,226.13,21.01
2,Consumer Staples,270.41,21.75
3,Energy,154.47,21.59
4,Financials,101.86,21.03
5,Health Care,61.78,22.32
6,Industrials,338.67,19.76
7,Information Technology,143.20,19.74


In [57]:
fig = px.scatter(df_plot,
                 x='volatility', 
                 y='rsi',
                 title='Volatility vs. RSI',
                 color='sector')
fig.show()